In [53]:
import sys
import os
import math
import random
import bisect
import pickle
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import urllib
import subprocess
import re
import tempfile
import itertools
import torch
import spacy
import amrlib
import penman

from typing import List, Tuple
from operator import itemgetter 
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification, BertTokenizer, BertForSequenceClassification
# import qgrid

BASE_DIR = os.path.abspath(os.getcwd()+'/../..')  # /home/gil/dev/NEBULA2/
os.chdir(os.getcwd()+'/../..')

from nebula_api.nebula_enrichment_api import *
from experts.common.RemoteAPIUtility import RemoteAPIUtility
from nebula_api.vlmapi import VLM_API
from nebula_api.atomic2020.comet_enrichment_api import *
from nebula_api.canonisation_api import CANON_API
import nebula_api.playground_api as pg_api

In [2]:
nre = NRE_API()
api = RemoteAPIUtility()
vlm = VLM_API()
ascore = CANON_API()
# comet = Comet("/app/NEBULA2/nebula_api/atomic2020/comet-atomic_2020_BART")
# stog = amrlib.load_stog_model(model_dir="/app/NEBULA2/models/model_stog")
# gtos = amrlib.load_gtos_model(model_dir="/app/NEBULA2/models/model_gtos")
# model_name = "Alireza1044/albert-base-v2-cola" 


# Download cola model
# cola_model = AutoModelForSequenceClassification.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

INFO:tensorflow:Restoring parameters from /app/NEBULA2/nebula_api/mdmmt_api/ckpts/vggish_model.ckpt


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.

Available class names: ['clip_vit', 'clip_rn', 'mdmmt_max', 'mdmmt_mean', 'mdmmt_legacy']


In [3]:
def flatten(lst): return [x for l in lst for x in l]

def compute_batch_scores(video_emb: torch.Tensor, texts: List[str], normalize=True, **kwargs) -> List[float]:    
    emb_batch = vlm.encode_text(texts, **kwargs)
    if type(emb_batch) == list:
        emb_batch = torch.stack(emb_batch,axis=0)
    if normalize:
        video_emb = (video_emb / video_emb.norm(2)).squeeze()           # This should be done in vlm
        # print("normalized video norm: {}".format(video_emb.norm(2)))
        n = (emb_batch * emb_batch).sum(axis=1).sqrt()
        emb_batch = emb_batch / n.unsqueeze(1).expand_as(emb_batch)
        # print("normalized text norms:")
        # for emb in emb_batch:
        #     print(emb.norm(2))                        
    return (video_emb.unsqueeze(0).expand_as(emb_batch)*emb_batch).sum(dim=1).cpu().numpy()


def compute_concat_score(image_emb: torch.Tensor, texts: List[str], join_on=',') -> float:
    combined_text = ""
    for t in [x.strip() for x in texts]:
        if t[-1]=='.':
            t = t[:-1]       
        t+=join_on
        t+=' '
        combined_text+=t
    print("Combined: "+combined_text)
    return torch.matmul(image_emb,mdmmt.encode_text(combined_text.strip()) )       

In [62]:
query = 'FOR doc IN s1_lsmdc_500 RETURN doc'
cursor = nre.db.aql.execute(query)
all_docs = sorted(list(cursor),key=lambda x:x['idx'])

In [63]:
[x['idx'] for x in all_docs if not x['groundtruth']]

[167, 201, 202, 203, 205]

In [64]:
prompt_template = "Original: {}\nCandidates: {}\n\n###\n\n"
processed_docs = []
for doc in all_docs:
    if not doc['groundtruth'] or doc['base_choice'] == None: 
        continue
    base = doc['captions'][doc['base_choice']] if type(doc['base_choice']) == int else doc['base_choice']
    prompt = prompt_template.format(base,'; '.join(doc['experts']))
    completion = doc['groundtruth']
    processed_docs.append({'prompt': prompt, 'completion': completion})

In [57]:
processed_docs

[{'prompt': 'Original: A woman with a strange face and strange eyes.\nCandidates: crying; wet; middle-aged\n\n###\n\n',
  'completion': 'A middle-aged woman with a strange face and wet eyes is crying.'},
 {'prompt': 'Original: A door with a clock on it in front of a window.\nCandidates: metal doorknob; white; wooden\n\n###\n\n',
  'completion': 'A wooden door with a metal doorknob on it in front of a window'},
 {'prompt': 'Original: A man standing in a dark room with a light on.\nCandidates: pijamas; foggy; bewildered\n\n###\n\n',
  'completion': 'A man in pijamas is standing in a foggy room with the light on, looking bewildered.'},
 {'prompt': 'Original: A girl with blue hair standing next to a boy.\nCandidates: young; bedroom; night lamp\n\n###\n\n',
  'completion': 'A young girl with blue hair is standing in a bedroom next to a boy, with a night lamp behind her.'},
 {'prompt': 'Original: A man is standing in front of a house with a blender.\nCandidates: yellow shirt; clean-shaven; g

In [65]:
len(processed_docs)

480

In [59]:
with open('/app/lsmdc_500_ft_data.json', 'w') as outfile:
    json.dump(processed_docs, outfile,indent=4)